## Analyse annual tides

In [ ]:
using DataFrames
using StatsBase
using XLSX
using Printf

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

excel_directory = ".\\" 
excel_file = excel_directory*"/annual_tides.xlsx"

work_sheets = XLSX.sheetnames(XLSX.readxlsx(excel_file))

global all_sites = DataFrame(XLSX.readtable(excel_file,1,))

# Get a list of sites
sites = names(all_sites[:,2:end])

# Get list of years to be processed
years = all_sites[:,1]

# calculate Mean and Stdev for each site
means = []; stdevs = []

for site_name in sites

    push!(means,mean(skipmissing(all_sites[:, site_name])))
    push!(stdevs,std(skipmissing(all_sites[:, site_name])))

end

sites = names(all_sites[:,2:end]);

years = all_sites.Year
global site_name = ""

for site_name in sites

    global other_sites = filter(x->x!=site_name, sites)    
    println("\n\n",site_name)
    print("     ")
    map(x -> @printf(" %5s ",x[1:5]), other_sites)[1]
    print(" Mean")
    
    site_total = []
    global site_means = []
    
    global matrix = Matrix{Float64}(undef, length(sites)-1, 0)
    
    for a_year in years
    
        print("\n",a_year,' ')
        global aa = []

        # calculate mean level of site
        global site_mean = means[findall(x->x==site_name, sites)]
        
        global all_sites_vals = []
        
        for xxx in other_sites

            # calculate mean level of other site
            other_site_mean = means[findall(x->x==xxx, sites)]

            # subtract other sites value from site value and subtract difference of means
            aa = all_sites[all_sites.Year .== a_year, xxx] - all_sites[all_sites.Year .== a_year, site_name];
            bb = other_site_mean - site_mean
            global cc = abs.(aa-bb)
            
            try
                @printf("%6.3f ", cc[1])
                push!(all_sites_vals,cc[1])                
            catch
                print(" ----- ")
                push!(all_sites_vals, missing)
            end

        end
        
        try        
            @printf("%6.3f ", mean(skipmissing(all_sites_vals)))
        catch
            print(" ----- ")
        end
        
        matrix = hcat(matrix,all_sites_vals)
        
        try            
            push!(site_means,mean(skipmissing(all_sites_vals)))
            @printf("%6.3f", mean.(skipmissing.(eachrow(all_sites_vals))))
        catch
            print(" ")
        end

    end
    
    print("\n     ")
    global means_x = mean.(skipmissing.(eachrow(matrix)))
    map(x -> @printf("%6.3f ",x), means_x)[1]
    @printf("%6.3f ",mean(site_means))
    print("\n     ")
    map(x -> @printf("%6.3f ",x), means[findall(x->x==site_name, sites)] .- means[1:end .!= findall(x->x==site_name, sites)])[1]
    @printf("%6.3f %6.3f",std(site_means),mean(site_means)+2*std(site_means))
   
end

In [ ]:
using DataFrames
using StatsBase
using XLSX
using Printf

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

excel_directory = ".\\" 
excel_file = excel_directory*"/annual_tides.xlsx"

work_sheets = XLSX.sheetnames(XLSX.readxlsx(excel_file))

global all_sites = DataFrame(XLSX.readtable(excel_file,1,))

# Get a list of sites
sites = names(all_sites[:,2:end])

# Get list of years to be processed
years = all_sites[:,1]

# calculate Mean and Stdev for each site
means = []; stdevs = []

for site_name in sites

    push!(means,mean(skipmissing(all_sites[:, site_name])))
    push!(stdevs,std(skipmissing(all_sites[:, site_name])))

end

sites = names(all_sites[:,2:end]);

global years = all_sites.Year
global site_name = ""

global excel_out_file = "annual_tides_output.xlsx"

XLSX.openxlsx("annual_tides_output.xlsx", mode="w") do xf
    
    for site_name in sites

        global other_sites = filter(x->x!=site_name, sites)    
        println("\n\n",site_name)
        print("     ")
        map(x -> @printf(" %5s ",x[1:5]), other_sites)[1]
        print(" Mean")

        site_total = []
        global site_means = []
        global mean_vals_total = []

        global matrix = Matrix{Float64}(undef, length(sites)-1, 0)

        # Create a df to hold the output data
        col_names = [["Date"]; other_sites; ["Mean"]]
        df = DataFrame([[] for _ = col_names] , col_names)

#        years = Int64.(years)

        for a_year in years

            print("\n",a_year,' ')
            global aa = []

            # calculate mean level of site
            global site_mean = means[findall(x->x==site_name, sites)]

            global all_sites_vals = []

            for xxx in other_sites

                # calculate mean level of other site
                other_site_mean = means[findall(x->x==xxx, sites)]

                # subtract other sites value from site value and subtract difference of means
                aa = all_sites[all_sites.Year .== a_year, xxx] - all_sites[all_sites.Year .== a_year, site_name];
                bb = other_site_mean - site_mean
                global cc = abs.(aa-bb)

                try
                    @printf("%6.3f ", cc[1])
                    push!(all_sites_vals,cc[1])                
                catch
                    print(" ----- ")
                    push!(all_sites_vals, missing)
                end

            end

            try        
                @printf("%6.3f ", mean(skipmissing(all_sites_vals)))
                global mean_val = mean(skipmissing(all_sites_vals))
            catch
                print(" ----- ")
                global mean_val = missing
            end

            push!(df,[[a_year]; all_sites_vals; [mean_val]])
            push!(mean_vals_total,mean_val)

            matrix = hcat(matrix,all_sites_vals)

            try            
                push!(site_means,mean(skipmissing(all_sites_vals)))
                @printf("%6.3f", mean.(skipmissing.(eachrow(all_sites_vals))))
            catch
                print(" ")
            end

        end

        print("\n     ")
        global means_x = mean.(skipmissing.(eachrow(matrix)))

        push!(df,[[""]; means_x; [mean(skipmissing(site_means))]])

        map(x -> @printf("%6.3f ",x), means_x)[1]
        @printf("%6.3f ",mean(site_means))

    #    push!(df,[[""]; all_sites_vals; [mean_val]])
        print("\n     ")
        map(x -> @printf("%6.3f ",x), means[findall(x->x==site_name, sites)] .- means[1:end .!= findall(x->x==site_name, sites)])[1]
        @printf("%6.3f %6.3f",std(site_means),mean(site_means)+2*std(site_means))
           push!(df,[[""]; means[findall(x->x==site_name, sites)] .- means[1:end .!= findall(x->x==site_name, sites)]; [std(skipmissing(site_means))]])


        sheet_name = site_name
        sheet = XLSX.addsheet!(xf, sheet_name)
        XLSX.writetable!(sheet, df)

    end

end


In [ ]:
work_sheets